# ----------------------

In [ ]:
import winreg
import subprocess
import json
import os

def explore_speech_registry():
    # Lista de rutas donde Windows almacena información de voz
    registry_paths = [
        (r"SOFTWARE\Microsoft\Speech_OneCore\Voices\Tokens", "Speech_OneCore"),
        (r"SOFTWARE\Microsoft\Speech Server\v11.0\Voices", "Speech Server"),
        (r"SOFTWARE\Microsoft\Speech\Voices\Tokens", "Speech"),
        (r"SOFTWARE\Microsoft\Windows NT\CurrentVersion\Speech\Voices", "Windows NT Speech"),
        (r"SOFTWARE\WOW6432Node\Microsoft\Speech\Voices\Tokens", "Speech WOW64")
    ]
    
    voices_found = {}
    
    for path, section in registry_paths:
        print(f"\nBuscando en {section}:")
        try:
            # Intentar abrir la clave con acceso de 64 bits
            key = winreg.OpenKey(winreg.HKEY_LOCAL_MACHINE, path, 0, 
                               winreg.KEY_READ | winreg.KEY_WOW64_64KEY)
            
            i = 0
            while True:
                try:
                    # Obtener subclave (token de voz)
                    voice_name = winreg.EnumKey(key, i)
                    voice_key = winreg.OpenKey(key, voice_name)
                    
                    print(f"\nVoz encontrada: {voice_name}")
                    
                    # Obtener todos los valores disponibles
                    j = 0
                    voice_data = {}
                    while True:
                        try:
                            value_name, value_data, _ = winreg.EnumValue(voice_key, j)
                            voice_data[value_name] = value_data
                            print(f"  {value_name} = {value_data}")
                            j += 1
                        except WindowsError:
                            break
                    
                    voices_found[voice_name] = voice_data
                    i += 1
                    winreg.CloseKey(voice_key)
                    
                except WindowsError:
                    break
            
            winreg.CloseKey(key)
            
        except WindowsError as e:
            print(f"No se pudo acceder a {section}: {e}")
    
    return voices_found

# Función adicional para buscar paquetes de voz instalados
def check_installed_voice_packages():
    print("\nBuscando paquetes de voz instalados:")
    try:
        result = subprocess.run(
            ["powershell", "-Command", 
             "Get-AppxPackage | Where-Object {$_.Name -like '*Speech*' -or $_.Name -like '*TTS*'}"],
            capture_output=True,
            text=True,
            encoding='cp1252'
        )
        print(result.stdout)
    except Exception as e:
        print(f"Error al buscar paquetes: {e}")

def main():
    print("Analizando voces instaladas en el sistema...")
    voices = explore_speech_registry()
    check_installed_voice_packages()
    
    # Guardar los resultados en un archivo para análisis
    try:
        with open('voces_encontradas.json', 'w', encoding='utf-8') as f:
            json.dump(voices, f, indent=2, ensure_ascii=False)
        print("\nLos resultados se han guardado en 'voces_encontradas.json'")
    except Exception as e:
        print(f"Error al guardar resultados: {e}")

if __name__ == "__main__":
    main()

In [ ]:
import subprocess

def get_uwp_voices():
    ps_script = '''
    # Cargar los ensamblados necesarios
    $asTaskGeneric = ([System.WindowsRuntimeSystemExtensions].GetMethods() | Where { 
        $_.Name -eq 'AsTask' -and $_.GetParameters().Count -eq 1 
    })[0].MakeGenericMethod([Windows.Media.SpeechSynthesis.VoiceInformation])

    try {
        # Obtener el objeto de sintetizador de voz
        $synth = New-Object Windows.Media.SpeechSynthesis.SpeechSynthesizer
        
        # Obtener las voces instaladas
        $voices = $synth.AllVoices
        
        foreach ($voice in $voices) {
            Write-Host "DisplayName:" $voice.DisplayName
            Write-Host "Id:" $voice.Id
            Write-Host "Language:" $voice.Language
            Write-Host "------------------------"
        }
        
    } catch {
        Write-Host "Error:" $_.Exception.Message
    }
    '''
    
    try:
        # Ejecutar PowerShell con los permisos adecuados
        result = subprocess.run(
            ["powershell", "-ExecutionPolicy", "Bypass", "-Command", 
             "$null = [Windows.Media.SpeechSynthesis.SpeechSynthesizer, Windows.Media.SpeechSynthesis, ContentType = WindowsRuntime]; " + ps_script],
            capture_output=True,
            text=True,
            encoding='cp1252'
        )
        
        if result.stderr:
            print("Errores:", result.stderr)
        
        if result.stdout:
            print("Voces encontradas:\n", result.stdout)
            
    except Exception as e:
        print(f"Error al ejecutar PowerShell: {e}")

# Alternativa usando acceso directo al sistema de archivos
def check_voice_files():
    print("\nBuscando archivos de voz instalados:")
    paths_to_check = [
        r"C:\Windows\Speech_OneCore\Engines\TTS",
        r"C:\Program Files\WindowsApps\Microsoft.Speech*",
        r"C:\Windows\Speech",
        r"C:\Windows\SysWOW64\Speech\Engines\TTS"
    ]
    
    for path in paths_to_check:
        try:
            result = subprocess.run(
                ["powershell", "-Command", f"Get-ChildItem -Path '{path}' -Recurse -ErrorAction SilentlyContinue"],
                capture_output=True,
                text=True,
                encoding='cp1252'
            )
            if result.stdout:
                print(f"\nArchivos encontrados en {path}:")
                print(result.stdout)
        except Exception as e:
            print(f"Error al buscar en {path}: {e}")

if __name__ == "__main__":
    print("Intentando obtener voces UWP...")
    get_uwp_voices()
    print("\nBuscando archivos de voz en el sistema...")
    check_voice_files()

In [ ]:
import subprocess

def get_voice_packages():
    ps_script = '''
    $packages = Get-AppxPackage -AllUsers | Where-Object {
        $_.Name -like "*Speech*" -or 
        $_.Name -like "*TTS*" -or 
        $_.Name -like "*TextToSpeech*"
    }
    
    foreach ($package in $packages) {
        Write-Host "Nombre del Paquete:" $package.Name
        Write-Host "Versión:" $package.Version
        Write-Host "Ubicación:" $package.InstallLocation
        Write-Host "------------------------"
    }
    
    # También buscar en el Manifest de Windows
    $manifestPath = "C:\Windows\SystemApps\Microsoft.Windows.NarratorQuickStart_*"
    if (Test-Path $manifestPath) {
        Get-ChildItem $manifestPath -Recurse | Where-Object {
            $_.Name -like "*Voice*" -or $_.Name -like "*Speech*"
        } | ForEach-Object {
            Write-Host "Archivo encontrado:" $_.FullName
        }
    }
    '''
    
    try:
        result = subprocess.run(
            ["powershell", "-Command", ps_script],
            capture_output=True,
            text=True,
            encoding='cp1252'
        )
        print(result.stdout)
    except Exception as e:
        print(f"Error: {e}")

def check_voice_keys():
    ps_script = '''
    $voicePaths = @(
        "HKLM:\SOFTWARE\Microsoft\Speech_OneCore\Voices\Tokens",
        "HKLM:\SOFTWARE\WOW6432Node\Microsoft\SPEECH\Voices\Tokens",
        "HKLM:\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Speech\Voices",
        "HKLM:\SOFTWARE\Microsoft\Speech\Voices\Tokens",
        "HKLM:\SOFTWARE\Microsoft\Speech Server\v11.0\Voices"
    )

    foreach ($path in $voicePaths) {
        Write-Host "`nChecking path: $path"
        if (Test-Path $path) {
            Get-ChildItem $path | ForEach-Object {
                $voiceName = $_.PSChildName
                Write-Host "`nVoice found: $voiceName"
                Get-ItemProperty $_.PSPath | ForEach-Object {
                    $_.PSObject.Properties | Where-Object { 
                        $_.Name -notlike "PS*" 
                    } | ForEach-Object {
                        Write-Host "  $($_.Name): $($_.Value)"
                    }
                }
            }
        }
    }
    '''
    
    try:
        result = subprocess.run(
            ["powershell", "-Command", ps_script],
            capture_output=True,
            text=True,
            encoding='cp1252'
        )
        print(result.stdout)
    except Exception as e:
        print(f"Error: {e}")

print("Buscando paquetes de voz instalados...")
get_voice_packages()

print("\nVerificando claves del registro relacionadas con voces...")
check_voice_keys()

In [ ]:
import subprocess

def scan_voice_packages():
    ps_script = '''
    # Buscar paquetes de voz específicos
    $voicePackages = @(
        "MicrosoftWindows.Voice*",
        "*Speech*",
        "*TTS*",
        "*TextToSpeech*"
    )

    foreach ($pattern in $voicePackages) {
        Write-Host "`nBuscando paquetes que coincidan con: $pattern"
        
        # Buscar en Program Files/WindowsApps
        $path = "C:\\Program Files\\WindowsApps\\$pattern"
        if (Test-Path $path) {
            Get-ChildItem $path -Directory | ForEach-Object {
                Write-Host "`nPaquete encontrado:"
                Write-Host "Nombre: $($_.Name)"
                Write-Host "Ruta: $($_.FullName)"
                
                # Buscar archivos de voz dentro del paquete
                Get-ChildItem $_.FullName -Recurse -File | Where-Object {
                    $_.Extension -in '.msix','.dat','.model'
                } | ForEach-Object {
                    Write-Host "  Archivo: $($_.Name)"
                }
            }
        }
        
        # Buscar en Windows\SystemApps
        $path = "C:\\Windows\\SystemApps\\$pattern"
        if (Test-Path $path) {
            Get-ChildItem $path -Directory | ForEach-Object {
                Write-Host "`nAplicación del sistema encontrada:"
                Write-Host "Nombre: $($_.Name)"
                Write-Host "Ruta: $($_.FullName)"
            }
        }
    }

    # Buscar específicamente los paquetes de Dalia y Jorge
    $specificPackages = @(
        "MicrosoftWindows.Voice.es-MX.Dalia*",
        "MicrosoftWindows.Voice.es-MX.Jorge*"
    )

    foreach ($package in $specificPackages) {
        Write-Host "`nBuscando paquete específico: $package"
        Get-AppxPackage $package | ForEach-Object {
            Write-Host "Nombre: $($_.Name)"
            Write-Host "Versión: $($_.Version)"
            Write-Host "Ubicación: $($_.InstallLocation)"
        }
    }
    '''

    try:
        result = subprocess.run(
            ["powershell", "-ExecutionPolicy", "Bypass", "-Command", ps_script],
            capture_output=True,
            text=True,
            encoding='cp1252'
        )
        print(result.stdout)
        
        if result.stderr:
            print("\nErrores encontrados:")
            print(result.stderr)
            
    except Exception as e:
        print(f"Error al ejecutar el script: {e}")

print("Escaneando paquetes de voz instalados...")
scan_voice_packages()

In [ ]:
import subprocess
import win32com.client
import os

def initialize_speech_with_uwp_voices():
    ps_script = '''
    # Construir la ruta al manifiesto del paquete
    $daliaPath = "C:\\Program Files\\WindowsApps\\MicrosoftWindows.Voice.es-MX.Dalia.1_1.0.3.0_x64__cw5n1h2txyewy"
    $jorgePath = "C:\\Program Files\\WindowsApps\\MicrosoftWindows.Voice.es-MX.Jorge.1_1.0.2.0_x64__cw5n1h2txyewy"
    
    # Intentar cargar las voces mediante Windows.Media.SpeechSynthesis
    Add-Type -AssemblyName System.Runtime
    
    try {
        # Registrar las voces en el sistema
        $regPath = "HKLM:\\SOFTWARE\\Microsoft\\Speech_OneCore\\Voices\\Tokens"
        
        # Para Dalia
        $daliaKey = "MSTTS_V110_esMX_DaliaM"
        if (!(Test-Path "$regPath\\$daliaKey")) {
            New-Item -Path "$regPath\\$daliaKey" -Force
            Set-ItemProperty -Path "$regPath\\$daliaKey" -Name "Path" -Value "$daliaPath"
            Set-ItemProperty -Path "$regPath\\$daliaKey" -Name "VoicePath" -Value "$daliaPath"
        }
        
        # Para Jorge
        $jorgeKey = "MSTTS_V110_esMX_JorgeM"
        if (!(Test-Path "$regPath\\$jorgeKey")) {
            New-Item -Path "$regPath\\$jorgeKey" -Force
            Set-ItemProperty -Path "$regPath\\$jorgeKey" -Name "Path" -Value "$jorgePath"
            Set-ItemProperty -Path "$regPath\\$jorgeKey" -Name "VoicePath" -Value "$jorgePath"
        }
        
        Write-Host "Voces registradas correctamente"
        
    } catch {
        Write-Host "Error al registrar las voces: $_"
    }
    '''

    try:
        # Ejecutar con privilegios elevados
        result = subprocess.run(
            ["powershell", "-ExecutionPolicy", "Bypass", "-Command", ps_script],
            capture_output=True,
            text=True,
            encoding='cp1252'
        )
        print(result.stdout)
        
        if result.stderr:
            print("\nErrores encontrados:")
            print(result.stderr)
            
    except Exception as e:
        print(f"Error al ejecutar el script: {e}")

def list_available_voices():
    try:
        # Intentar obtener las voces usando SAPI
        speaker = win32com.client.Dispatch("SAPI.SpVoice")
        voices = speaker.GetVoices()
        
        print("\nVoces disponibles:")
        for voice in voices:
            try:
                desc = voice.GetDescription()
                print(f"- {desc}")
            except:
                print("Error al obtener descripción de voz")
                
    except Exception as e:
        print(f"Error al listar voces: {e}")

print("Inicializando voces...")
initialize_speech_with_uwp_voices()

print("\nListando voces disponibles...")
list_available_voices()

In [ ]:
import subprocess

def try_access_uwp_voices():
    ps_script = '''
    # Intentar cargar el ensamblado de Windows.Media.SpeechSynthesis
    $null = [Windows.Media.SpeechSynthesis.SpeechSynthesizer, Windows.Media.SpeechSynthesis, ContentType = WindowsRuntime]
    
    try {
        # Crear una nueva instancia del sintetizador
        $synth = New-Object Windows.Media.SpeechSynthesis.SpeechSynthesizer
        
        # Intentar obtener las voces instaladas
        $voices = $synth.AllVoices
        
        Write-Host "Voces UWP encontradas:"
        foreach ($voice in $voices) {
            Write-Host ("Nombre: " + $voice.DisplayName)
            Write-Host ("ID: " + $voice.Id)
            Write-Host ("Idioma: " + $voice.Language)
            Write-Host "------------------------"
        }
        
    } catch {
        Write-Host "Error accediendo a las voces UWP: $_"
        
        # Intentar un método alternativo usando WinRT
        try {
            $rtClass = [Windows.Media.SpeechSynthesis.SpeechSynthesizer, Windows.Media.SpeechSynthesis, ContentType = WindowsRuntime]
            Write-Host "Clase WinRT cargada correctamente"
            
            # Intentar listar las voces instaladas
            $voicePath = "C:\\Program Files\\WindowsApps\\MicrosoftWindows.Voice.es-MX.*"
            Get-ChildItem $voicePath -Directory | ForEach-Object {
                Write-Host "Paquete de voz encontrado:"
                Write-Host $_.FullName
            }
            
        } catch {
            Write-Host "Error con WinRT: $_"
        }
    }
    '''
    
    try:
        result = subprocess.run(
            ["powershell", "-ExecutionPolicy", "Bypass", "-Command", ps_script],
            capture_output=True,
            text=True,
            encoding='cp1252'
        )
        print(result.stdout)
        
        if result.stderr:
            print("\nErrores encontrados:")
            print(result.stderr)
            
    except Exception as e:
        print(f"Error ejecutando el script: {e}")

print("Intentando acceder a las voces UWP sin privilegios de administrador...")
try_access_uwp_voices()

In [ ]:
import subprocess

def get_narrator_voices():
    ps_script = '''
    Add-Type -AssemblyName UIAutomationClient
    Add-Type -AssemblyName System.Windows.Forms

    try {
        # Buscar los paquetes de voz instalados
        $voicePackages = Get-AppxPackage -Name "MicrosoftWindows.Voice*"
        
        Write-Host "Paquetes de voz encontrados:"
        foreach ($package in $voicePackages) {
            Write-Host "Nombre:" $package.Name
            Write-Host "Versión:" $package.Version
            Write-Host "Ubicación:" $package.InstallLocation
            Write-Host "------------------------"
            
            # Buscar archivos de voz en la ubicación del paquete
            if (Test-Path $package.InstallLocation) {
                $files = Get-ChildItem -Path $package.InstallLocation -Recurse -File |
                    Where-Object { $_.Extension -in '.dat','.model' }
                
                foreach ($file in $files) {
                    Write-Host "Archivo de voz:" $file.Name
                }
            }
        }
        
        # Buscar en el directorio de voces de OneCore
        $onecorePath = "C:\\Windows\\Speech_OneCore\\Engines\\TTS"
        if (Test-Path $onecorePath) {
            Write-Host "`nVoces OneCore encontradas:"
            Get-ChildItem -Path $onecorePath -Directory | ForEach-Object {
                Write-Host "Idioma:" $_.Name
                
                # Listar archivos de voz
                Get-ChildItem -Path $_.FullName -File | Where-Object {
                    $_.Name -like "M*.APM" -or $_.Name -like "M*.INI"
                } | ForEach-Object {
                    Write-Host "  Archivo:" $_.Name
                }
            }
        }

    } catch {
        Write-Host "Error:" $_
    }
    '''
    
    try:
        result = subprocess.run(
            ["powershell", "-ExecutionPolicy", "Bypass", "-Command", ps_script],
            capture_output=True,
            text=True,
            encoding='cp1252'
        )
        print(result.stdout)
        
        if result.stderr:
            print("\nErrores encontrados:")
            print(result.stderr)
            
    except Exception as e:
        print(f"Error al ejecutar el script: {e}")

print("Buscando información de voces instaladas...")
get_narrator_voices()

In [3]:
import win32com.client

class TextToSpeech:
    def __init__(self, default_pitch=0, default_rate=3, default_volume=100):
        self.default_pitch = default_pitch
        self.default_rate = default_rate
        self.default_volume = default_volume
        
    def speak_result(self, text, pitch=None, rate=None, volume=None):
        pitch = pitch if pitch is not None else self.default_pitch
        rate = rate if rate is not None else self.default_rate
        volume = volume if volume is not None else self.default_volume
        
        speaker = win32com.client.Dispatch("SAPI.SpVoice")
        voices = speaker.GetVoices()
        speaker.Voice = voices[2]
        
        xml_text = f"""
        <pitch middle="{pitch}">
            <rate speed="{rate}">
                <volume level="{volume}">
                    {text}
                </volume>
            </rate>
        </pitch>
        """
        
        speaker.Speak(xml_text)

In [ ]:
tts = TextToSpeech()

result = "Hola. Esta es una prueba de síntesis de voz desde python, usando las voces de windows."
print(result)
tts.speak_result(result, pitch=0, rate=2, volume=100)

In [ ]:
import win32com.client
import subprocess

class TextToSpeech:
    def __init__(self, default_pitch=0, default_rate=3, default_volume=100):
        self.default_pitch = default_pitch
        self.default_rate = default_rate
        self.default_volume = default_volume
        self._initialize_onecore_voices()
        
    def _initialize_onecore_voices(self):
        # Intenta registrar las voces OneCore para SAPI
        ps_script = '''
        try {
            # Verificar si la voz ya está registrada
            $regPath = "HKCU:\\SOFTWARE\\Microsoft\\Speech_OneCore\\Voices\\Tokens"
            $voiceName = "MSTTS_V110_esMX_DaliaM"
            
            # Obtener la ruta del paquete de Dalia
            $daliaPackage = Get-AppxPackage "MicrosoftWindows.Voice.es-MX.Dalia.1"
            if ($daliaPackage) {
                $daliaPath = $daliaPackage.InstallLocation
                
                # Intentar registrar en HKEY_CURRENT_USER en lugar de HKEY_LOCAL_MACHINE
                if (!(Test-Path "$regPath\\$voiceName")) {
                    New-Item -Path "$regPath\\$voiceName" -Force | Out-Null
                    New-ItemProperty -Path "$regPath\\$voiceName" -Name "Path" -Value $daliaPath -PropertyType String -Force | Out-Null
                    New-ItemProperty -Path "$regPath\\$voiceName" -Name "VoicePath" -Value $daliaPath -PropertyType String -Force | Out-Null
                }
            }
        } catch {
            Write-Host "Info: No se pudo registrar la voz OneCore (esto es normal si no hay permisos de admin)"
        }
        '''
        
        try:
            subprocess.run(["powershell", "-Command", ps_script], 
                         capture_output=True, text=True, encoding='cp1252')
        except Exception:
            pass

    def get_voice_by_name(self, speaker, name):
        voices = speaker.GetVoices()
        for i in range(voices.Count):
            if name.lower() in voices.Item(i).GetDescription().lower():
                return voices.Item(i)
        return None

    def speak_result(self, text, pitch=None, rate=None, volume=None, voice_name="Dalia"):
        pitch = pitch if pitch is not None else self.default_pitch
        rate = rate if rate is not None else self.default_rate
        volume = volume if volume is not None else self.default_volume
        
        speaker = win32com.client.Dispatch("SAPI.SpVoice")
        
        # Intentar encontrar la voz por nombre
        voice = self.get_voice_by_name(speaker, voice_name)
        
        # Si no se encuentra Dalia, intentar con Sabina como respaldo
        if voice is None:
            voice = self.get_voice_by_name(speaker, "Sabina")
            print(f"No se encontró la voz {voice_name}, usando Sabina como respaldo")
        
        if voice:
            speaker.Voice = voice
        
        xml_text = f"""
        <pitch middle="{pitch}">
            <rate speed="{rate}">
                <volume level="{volume}">
                    {text}
                </volume>
            </rate>
        </pitch>
        """
        
        speaker.Speak(xml_text)

    def list_available_voices(self):
        """Lista todas las voces disponibles"""
        speaker = win32com.client.Dispatch("SAPI.SpVoice")
        voices = speaker.GetVoices()
        print("\nVoces disponibles:")
        for i in range(voices.Count):
            print(f"{i}: {voices.Item(i).GetDescription()}")

# Uso de la clase
try:
    # Crear instancia
    tts = TextToSpeech()
    
    # Listar voces disponibles
    tts.list_available_voices()
    
    # Prueba de voz
    result = "Hola. Esta es una prueba de síntesis de voz desde python, usando las voces de windows."
    print("\nReproduciendo mensaje de prueba:")
    print(result)
    
    # Intentar usar Dalia (con fallback a Sabina si no está disponible)
    tts.speak_result(result, pitch=0, rate=1, volume=100, voice_name="Dalia")
    
except Exception as e:
    print(f"Error: {str(e)}")

In [ ]:
import win32com.client
import subprocess

class TextToSpeech:
    def __init__(self, default_pitch=0, default_rate=3, default_volume=100):
        self.default_pitch = default_pitch
        self.default_rate = default_rate
        self.default_volume = default_volume

    def _try_uwp_voice(self, text):
        ps_script = '''
        Add-Type -AssemblyName System.Runtime.WindowsRuntime
        Add-Type -AssemblyName System.Runtime
        
        try {
            # Crear el sintetizador
            $synth = New-Object -TypeName Windows.Media.SpeechSynthesis.SpeechSynthesizer
            
            # Buscar la voz de Dalia
            $dalia = $synth.AllVoices | Where-Object { $_.DisplayName -like "*Dalia*" }
            
            if ($dalia) {
                $synth.Voice = $dalia
                # Sintetizar el texto
                $stream = $synth.SynthesizeTextToStreamAsync($args[0]).GetAwaiter().GetResult()
                
                # Reproducir el audio
                $player = New-Object System.Media.SoundPlayer
                $player.Stream = $stream.AsStream()
                $player.Play()
                
                return $true
            } else {
                return $false
            }
        } catch {
            Write-Host "Error UWP:" $_
            return $false
        }
        '''
        
        try:
            result = subprocess.run(
                ["powershell", "-Command", ps_script, text],
                capture_output=True,
                text=True
            )
            return "Error UWP" not in result.stdout
        except Exception as e:
            print(f"Error al intentar usar voz UWP: {e}")
            return False

    def speak_result(self, text, pitch=None, rate=None, volume=None):
        pitch = pitch if pitch is not None else self.default_pitch
        rate = rate if rate is not None else self.default_rate
        volume = volume if volume is not None else self.default_volume

        # Primero intentar usar la voz UWP (Dalia)
        if not self._try_uwp_voice(text):
            print("No se pudo usar Dalia, usando Sabina como respaldo...")
            # Si falla, usar SAPI con Sabina
            speaker = win32com.client.Dispatch("SAPI.SpVoice")
            voices = speaker.GetVoices()
            
            # Buscar Sabina
            for i in range(voices.Count):
                if "Sabina" in voices.Item(i).GetDescription():
                    speaker.Voice = voices.Item(i)
                    break
            
            xml_text = f"""
            <pitch middle="{pitch}">
                <rate speed="{rate}">
                    <volume level="{volume}">
                        {text}
                    </volume>
                </rate>
            </pitch>
            """
            
            speaker.Speak(xml_text)

    def list_available_voices(self):
        """Lista todas las voces disponibles (tanto SAPI como UWP)"""
        ps_script = '''
        # Listar voces SAPI
        $sapi = New-Object -ComObject SAPI.SpVoice
        $sapi.GetVoices() | ForEach-Object {
            Write-Host "SAPI:" $_.GetDescription()
        }
        
        try {
            # Listar voces UWP
            $synth = New-Object -TypeName Windows.Media.SpeechSynthesis.SpeechSynthesizer
            $synth.AllVoices | ForEach-Object {
                Write-Host "UWP:" $_.DisplayName
            }
        } catch {
            Write-Host "No se pudieron listar las voces UWP"
        }
        '''
        
        try:
            result = subprocess.run(
                ["powershell", "-Command", ps_script],
                capture_output=True,
                text=True,
                encoding='cp1252'
            )
            print("\nVoces disponibles:")
            print(result.stdout)
        except Exception as e:
            print(f"Error al listar voces: {e}")

# Ejemplo de uso
tts = TextToSpeech()
tts.list_available_voices()

result = "Hola. Esta es una prueba de síntesis de voz desde python, usando las voces de windows."
print("\nReproduciendo mensaje de prueba:")
print(result)
tts.speak_result(result, pitch=0, rate=1, volume=100)

In [ ]:
import win32com.client
import subprocess
import time

class TextToSpeech:
    def __init__(self, default_pitch=0, default_rate=3, default_volume=100):
        self.default_pitch = default_pitch
        self.default_rate = default_rate
        self.default_volume = default_volume
        
    def _speak_with_narrator(self, text):
        ps_script = '''
        Add-Type -TypeDefinition @"
        using System;
        using System.Runtime.InteropServices;

        public class NarratorAPI {
            [DllImport("Narrator.exe", CharSet = CharSet.Unicode)]
            public static extern void NarratorSpeak(string text);
        }
"@

        # Configurar el Narrador usando las teclas de Windows
        $wshell = New-Object -ComObject WScript.Shell

        # Abrir la configuración del Narrador
        $wshell.SendKeys("^%{ENTER}")  # Windows+Ctrl+Enter
        Start-Sleep -Seconds 1
        
        # Intentar cambiar a la voz de Dalia si está disponible
        $wshell.SendKeys("{TAB}")
        Start-Sleep -Milliseconds 500
        $wshell.SendKeys("d")  # Seleccionar Dalia
        Start-Sleep -Milliseconds 500

        # Narrar el texto
        [System.Windows.Forms.SendKeys]::SendWait($args[0])
        Start-Sleep -Seconds 2

        # Cerrar el Narrador
        $wshell.SendKeys("^%{ENTER}")
        '''
        
        try:
            subprocess.run(
                ["powershell", "-Command", ps_script, text],
                capture_output=True,
                text=True,
                encoding='cp1252'
            )
            return True
        except Exception as e:
            print(f"Error al usar el Narrador: {e}")
            return False

    def speak_result(self, text, pitch=None, rate=None, volume=None, use_narrator=True):
        if use_narrator:
            success = self._speak_with_narrator(text)
            if not success:
                print("No se pudo usar el Narrador, usando Sabina como respaldo...")
                self._speak_with_sapi(text, pitch, rate, volume)
        else:
            self._speak_with_sapi(text, pitch, rate, volume)

    def _speak_with_sapi(self, text, pitch=None, rate=None, volume=None):
        pitch = pitch if pitch is not None else self.default_pitch
        rate = rate if rate is not None else self.default_rate
        volume = volume if volume is not None else self.default_volume
        
        speaker = win32com.client.Dispatch("SAPI.SpVoice")
        voices = speaker.GetVoices()
        
        # Buscar Sabina como respaldo
        for i in range(voices.Count):
            if "Sabina" in voices.Item(i).GetDescription():
                speaker.Voice = voices.Item(i)
                break
        
        xml_text = f"""
        <pitch middle="{pitch}">
            <rate speed="{rate}">
                <volume level="{volume}">
                    {text}
                </volume>
            </rate>
        </pitch>
        """
        
        speaker.Speak(xml_text)

    def list_available_voices(self):
        print("\nVoces SAPI disponibles:")
        speaker = win32com.client.Dispatch("SAPI.SpVoice")
        voices = speaker.GetVoices()
        for i in range(voices.Count):
            print(f"{i}: {voices.Item(i).GetDescription()}")
        
        print("\nVoces del Narrador:")
        print("* Las voces del Narrador solo están disponibles cuando se usa use_narrator=True")

# Ejemplo de uso
if __name__ == "__main__":
    tts = TextToSpeech()
    tts.list_available_voices()
    
    result = "Hola. Esta es una prueba de síntesis de voz desde python, usando las voces de windows."
    print("\nReproduciendo mensaje de prueba:")
    print(result)
    
    # Usar el Narrador (intentará usar Dalia)
    print("\nIntentando usar el Narrador (Dalia):")
    tts.speak_result(result, use_narrator=True)
    
    time.sleep(2)  # Esperar un poco
    
    # Usar SAPI (usará Sabina)
    print("\nUsando SAPI (Sabina):")
    tts.speak_result(result, use_narrator=False)

In [ ]:
import win32com.client
import subprocess
import sys
import time
from ctypes import windll

class TextToSpeech:
    def __init__(self, default_pitch=0, default_rate=3, default_volume=100):
        self.default_pitch = default_pitch
        self.default_rate = default_rate
        self.default_volume = default_volume

    def _try_dalia_with_uiautomation(self, text):
        ps_script = '''
        Add-Type -AssemblyName UIAutomationClient
        Add-Type -AssemblyName System.Windows.Forms
        
        function Invoke-Narrator {
            param($TextToSpeak)
            
            # Iniciar el Narrador
            $shell = New-Object -ComObject WScript.Shell
            $shell.Run("ms-settings:easeofaccess-narrator")
            Start-Sleep -Seconds 2
            
            try {
                $automation = [System.Windows.Automation.AutomationElement]::RootElement
                
                # Buscar la configuración de voz
                $settingsWindow = $automation.FindFirst(
                    [System.Windows.Automation.TreeScope]::Children,
                    [System.Windows.Automation.PropertyCondition]::new(
                        [System.Windows.Automation.AutomationElement]::NameProperty,
                        "Narrator"
                    )
                )
                
                if ($settingsWindow) {
                    # Intentar configurar la voz a Dalia
                    $voiceCombo = $settingsWindow.FindFirst(
                        [System.Windows.Automation.TreeScope]::Descendants,
                        [System.Windows.Automation.PropertyCondition]::new(
                            [System.Windows.Automation.AutomationElement]::AutomationIdProperty,
                            "ComboBox_ChooseAVoice"
                        )
                    )
                    
                    if ($voiceCombo) {
                        $pattern = $voiceCombo.GetCurrentPattern([System.Windows.Automation.SelectionPattern]::Pattern)
                        $pattern.Select()
                        [System.Windows.Forms.SendKeys]::SendWait("Dalia")
                        Start-Sleep -Milliseconds 500
                        [System.Windows.Forms.SendKeys]::SendWait("{ENTER}")
                    }
                    
                    # Ahora intentar narrar el texto
                    [System.Windows.Forms.SendKeys]::SendWait("^{ENTER}")  # Activar el Narrador
                    Start-Sleep -Milliseconds 500
                    [System.Windows.Forms.SendKeys]::SendWait($TextToSpeak)
                    Start-Sleep -Seconds 2
                    [System.Windows.Forms.SendKeys]::SendWait("^{ENTER}")  # Desactivar el Narrador
                }
            }
            catch {
                Write-Host "Error: $_"
            }
            finally {
                # Cerrar la ventana de configuración
                Stop-Process -Name "SystemSettings" -ErrorAction SilentlyContinue
            }
        }
        
        Invoke-Narrator -TextToSpeak $args[0]
        '''
        
        try:
            result = subprocess.run(
                ["powershell", "-Command", ps_script, text],
                capture_output=True,
                text=True,
                encoding='cp1252'
            )
            return "Error:" not in result.stdout
        except Exception as e:
            print(f"Error al usar UI Automation: {e}")
            return False

    def speak_result(self, text, pitch=None, rate=None, volume=None, use_dalia=True):
        if use_dalia:
            success = self._try_dalia_with_uiautomation(text)
            if not success:
                print("No se pudo usar Dalia, usando Sabina como respaldo...")
                self._speak_with_sapi(text, pitch, rate, volume)
        else:
            self._speak_with_sapi(text, pitch, rate, volume)

    def _speak_with_sapi(self, text, pitch=None, rate=None, volume=None):
        pitch = pitch if pitch is not None else self.default_pitch
        rate = rate if rate is not None else self.default_rate
        volume = volume if volume is not None else self.default_volume
        
        speaker = win32com.client.Dispatch("SAPI.SpVoice")
        voices = speaker.GetVoices()
        
        # Buscar Sabina
        for i in range(voices.Count):
            if "Sabina" in voices.Item(i).GetDescription():
                speaker.Voice = voices.Item(i)
                break
        
        xml_text = f"""
        <pitch middle="{pitch}">
            <rate speed="{rate}">
                <volume level="{volume}">
                    {text}
                </volume>
            </rate>
        </pitch>
        """
        
        speaker.Speak(xml_text)

# Prueba
if __name__ == "__main__":
    tts = TextToSpeech()
    
    result = "Hola. Esta es una prueba de síntesis de voz desde python."
    print("\nReproduciendo mensaje de prueba:")
    print(result)
    
    # Intentar usar Dalia
    print("\nIntentando usar Dalia:")
    tts.speak_result(result, use_dalia=True)
    
    time.sleep(2)
    
    # Usar Sabina
    print("\nUsando Sabina:")
    tts.speak_result(result, use_dalia=False)

In [ ]:
import win32com.client

class TextToSpeech:
    def __init__(self, default_pitch=0, default_rate=3, default_volume=100):
        self.default_pitch = default_pitch
        self.default_rate = default_rate
        self.default_volume = default_volume
        
    def speak_result(self, text, pitch=None, rate=None, volume=None):
        pitch = pitch if pitch is not None else self.default_pitch
        rate = rate if rate is not None else self.default_rate
        volume = volume if volume is not None else self.default_volume
        
        speaker = win32com.client.Dispatch("SAPI.SpVoice")
        voices = speaker.GetVoices()
        
        print("\nVoces disponibles:")
        for i in range(voices.Count):
            print(f"{i}: {voices.Item(i).GetDescription()}")
        
        # Intentar encontrar Dalia
        dalia_found = False
        for i in range(voices.Count):
            if "Dalia" in voices.Item(i).GetDescription():
                speaker.Voice = voices.Item(i)
                dalia_found = True
                print("\nUsando voz de Dalia")
                break
        
        if not dalia_found:
            print("\nNo se encontró la voz de Dalia, usando Sabina como respaldo")
            # Usar Sabina como respaldo
            for i in range(voices.Count):
                if "Sabina" in voices.Item(i).GetDescription():
                    speaker.Voice = voices.Item(i)
                    break
        
        xml_text = f"""
        <pitch middle="{pitch}">
            <rate speed="{rate}">
                <volume level="{volume}">
                    {text}
                </volume>
            </rate>
        </pitch>
        """
        
        speaker.Speak(xml_text)

# Prueba
if __name__ == "__main__":
    tts = TextToSpeech()
    result = "Hola. Esta es una prueba de síntesis de voz desde python, usando las voces de windows."
    print("\nReproduciendo mensaje de prueba:")
    print(result)
    tts.speak_result(result, pitch=0, rate=1, volume=100)

In [ ]:
import win32com.client
import os

class TextToSpeech:
    def __init__(self, default_pitch=0, default_rate=3, default_volume=100):
        self.default_pitch = default_pitch
        self.default_rate = default_rate
        self.default_volume = default_volume
        
    def speak_result(self, text, pitch=None, rate=None, volume=None):
        pitch = pitch if pitch is not None else self.default_pitch
        rate = rate if rate is not None else self.default_rate
        volume = volume if volume is not None else self.default_volume
        
        try:
            # Intentar usar directamente el token de Dalia
            speaker = win32com.client.Dispatch("SAPI.SpVoice")
            tokenKey = "HKEY_CURRENT_USER\\SOFTWARE\\Microsoft\\Speech_OneCore\\Voices\\Tokens\\MSTTS_V110_esMX_DaliaM"
            speaker.Voice = speaker.GetVoices(f"Name=Microsoft Dalia;Key={tokenKey}").Item(0)
            
            xml_text = f"""
            <pitch middle="{pitch}">
                <rate speed="{rate}">
                    <volume level="{volume}">
                        {text}
                    </volume>
                </rate>
            </pitch>
            """
            
            speaker.Speak(xml_text)
            print("Usando voz de Dalia")
            
        except Exception as e:
            print(f"Error al usar Dalia: {str(e)}")
            print("Usando Sabina como respaldo...")
            
            # Usar Sabina como respaldo
            try:
                speaker = win32com.client.Dispatch("SAPI.SpVoice")
                voices = speaker.GetVoices()
                
                for i in range(voices.Count):
                    if "Sabina" in voices.Item(i).GetDescription():
                        speaker.Voice = voices.Item(i)
                        break
                
                xml_text = f"""
                <pitch middle="{pitch}">
                    <rate speed="{rate}">
                        <volume level="{volume}">
                            {text}
                        </volume>
                    </rate>
                </pitch>
                """
                
                speaker.Speak(xml_text)
                
            except Exception as e:
                print(f"Error al usar voz de respaldo: {str(e)}")

    def list_available_voices(self):
        print("\nVoces registradas:")
        try:
            speaker = win32com.client.Dispatch("SAPI.SpVoice")
            
            # Intentar mostrar la voz de Dalia
            try:
                tokenKey = "HKEY_CURRENT_USER\\SOFTWARE\\Microsoft\\Speech_OneCore\\Voices\\Tokens\\MSTTS_V110_esMX_DaliaM"
                dalia_voice = speaker.GetVoices(f"Name=Microsoft Dalia;Key={tokenKey}").Item(0)
                print("Dalia encontrada:", dalia_voice.GetDescription())
            except:
                print("Dalia no encontrada en el registro")
            
            # Mostrar todas las voces SAPI
            voices = speaker.GetVoices()
            for i in range(voices.Count):
                print(f"{i}: {voices.Item(i).GetDescription()}")
                
        except Exception as e:
            print(f"Error al listar voces: {str(e)}")

# Prueba
if __name__ == "__main__":
    tts = TextToSpeech()
    
    # Listar voces disponibles
    tts.list_available_voices()
    
    # Probar la síntesis
    result = "Hola. Esta es una prueba de síntesis de voz desde python, usando las voces de windows."
    print("\nReproduciendo mensaje de prueba:")
    print(result)
    tts.speak_result(result, pitch=0, rate=1, volume=100)

In [ ]:
import subprocess
import os
import tempfile

class TextToSpeech:
    def __init__(self):
        # Crear el archivo C# temporal
        self.cs_code = """
using System;
using System.Threading.Tasks;
using Windows.Media.SpeechSynthesis;
using System.IO;

public class VoiceSynthesizer
{
    public static async Task SynthesizeToFile(string text, string outputPath)
    {
        try
        {
            using (var synth = new SpeechSynthesizer())
            {
                // Intentar encontrar la voz de Dalia
                var voices = synth.AllVoices;
                foreach (var voice in voices)
                {
                    if (voice.DisplayName.Contains("Dalia"))
                    {
                        synth.Voice = voice;
                        break;
                    }
                }

                // Sintetizar el texto
                var stream = await synth.SynthesizeTextToStreamAsync(text);

                // Guardar el audio
                using (var fileStream = new FileStream(outputPath, FileMode.Create))
                {
                    using (var reader = new Windows.Storage.Streams.DataReader(stream))
                    {
                        await reader.LoadAsync((uint)stream.Size);
                        byte[] buffer = new byte[stream.Size];
                        reader.ReadBytes(buffer);
                        fileStream.Write(buffer, 0, buffer.Length);
                    }
                }
            }
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Error: {ex.Message}");
            throw;
        }
    }
}
"""
        # Guardar el código en un archivo temporal
        self.cs_file = os.path.join(tempfile.gettempdir(), "VoiceSynth.cs")
        with open(self.cs_file, "w") as f:
            f.write(self.cs_code)

    def speak_result(self, text):
        try:
            # Compilar y ejecutar el código C#
            output_wav = os.path.join(tempfile.gettempdir(), "output.wav")
            
            # Compilar
            compile_cmd = f"""
            Add-Type -Path "{self.cs_file}" -ReferencedAssemblies @(
                "System.Runtime.WindowsRuntime.dll",
                "Windows.Foundation.dll",
                "System.Runtime.InteropServices.WindowsRuntime.dll"
            )
            """
            
            # Ejecutar
            run_cmd = f"""
            [VoiceSynthesizer]::SynthesizeToFile("{text}", "{output_wav}")
            """
            
            subprocess.run(["powershell", "-Command", compile_cmd + run_cmd], 
                         capture_output=True, text=True)

            # Reproducir el archivo de audio usando PowerShell
            play_cmd = f"""
            $player = New-Object System.Media.SoundPlayer
            $player.SoundLocation = "{output_wav}"
            $player.Play()
            Start-Sleep -Seconds 2  # Esperar a que termine
            $player.Dispose()
            """
            subprocess.run(["powershell", "-Command", play_cmd])

        except Exception as e:
            print(f"Error: {str(e)}")
            # Usar SAPI como respaldo
            try:
                import win32com.client
                speaker = win32com.client.Dispatch("SAPI.SpVoice")
                for voice in speaker.GetVoices():
                    if "Sabina" in voice.GetDescription():
                        speaker.Voice = voice
                        break
                speaker.Speak(text)
            except Exception as e2:
                print(f"Error en respaldo: {str(e2)}")

# Prueba
if __name__ == "__main__":
    tts = TextToSpeech()
    text = "Hola. Esta es una prueba de síntesis de voz desde python, usando las voces de windows."
    print("\nReproduciendo mensaje de prueba:")
    print(text)
    tts.speak_result(text)

In [ ]:
import win32com.client

class TextToSpeech:
    def __init__(self, default_pitch=0, default_rate=3, default_volume=100):
        self.default_pitch = default_pitch
        self.default_rate = default_rate
        self.default_volume = default_volume
        
    def list_available_voices(self):
        try:
            speaker = win32com.client.Dispatch("SAPI.SpVoice")
            voices = speaker.GetVoices()
            print("\nVoces disponibles:")
            for i in range(voices.Count):
                print(f"{i}: {voices.Item(i).GetDescription()}")
        except Exception as e:
            print(f"Error al listar voces: {str(e)}")
            
    def speak_result(self, text, voice_name="Dalia", pitch=None, rate=None, volume=None):
        pitch = pitch if pitch is not None else self.default_pitch
        rate = rate if rate is not None else self.default_rate
        volume = volume if volume is not None else self.default_volume
        
        speaker = win32com.client.Dispatch("SAPI.SpVoice")
        voices = speaker.GetVoices()
        
        # Buscar la voz solicitada
        voice_found = False
        for i in range(voices.Count):
            if voice_name.lower() in voices.Item(i).GetDescription().lower():
                speaker.Voice = voices.Item(i)
                voice_found = True
                print(f"Usando voz de {voice_name}")
                break
        
        if not voice_found:
            print(f"No se encontró la voz de {voice_name}, usando Sabina como respaldo")
            # Usar Sabina como respaldo
            for i in range(voices.Count):
                if "Sabina" in voices.Item(i).GetDescription():
                    speaker.Voice = voices.Item(i)
                    break
        
        xml_text = f"""
        <pitch middle="{pitch}">
            <rate speed="{rate}">
                <volume level="{volume}">
                    {text}
                </volume>
            </rate>
        </pitch>
        """
        
        speaker.Speak(xml_text)

# Prueba
if __name__ == "__main__":
    tts = TextToSpeech()
    
    # Listar voces disponibles
    tts.list_available_voices()
    
    test_text = "Hola. Esta es una prueba de síntesis de voz desde python, usando las voces de windows."
    
    # Probar con Dalia
    print("\nProbando voz de Dalia:")
    tts.speak_result(test_text, voice_name="Dalia")
    
    # Probar con Jorge
    print("\nProbando voz de Jorge:")
    tts.speak_result(test_text, voice_name="Jorge")

In [ ]:
import subprocess
import win32com.client
import time

class TextToSpeech:
    def __init__(self):
        self.use_uwp = True
    
    def _try_uwp_voice(self, text, voice_name):
        ps_script = '''
        Add-Type -AssemblyName System.Runtime.WindowsRuntime
        Add-Type -AssemblyName System.Runtime

        try {
            $synth = New-Object Windows.Media.SpeechSynthesis.SpeechSynthesizer
            $voice = $synth.AllVoices | Where-Object { $_.DisplayName -like "*$($args[1])*" } | Select-Object -First 1
            
            if ($voice) {
                $synth.Voice = $voice
                [void]$synth.SynthesizeTextToStreamAsync($args[0]).AsTask().Wait()
                Write-Host "UWP_SUCCESS"
            } else {
                Write-Host "UWP_VOICE_NOT_FOUND"
            }
        } catch {
            Write-Host "UWP_ERROR: $_"
        }
        '''
        
        try:
            result = subprocess.run(
                ["powershell", "-Command", ps_script, text, voice_name],
                capture_output=True,
                text=True,
                encoding='cp1252'
            )
            return "UWP_SUCCESS" in result.stdout
        except Exception as e:
            print(f"Error en UWP: {e}")
            return False

    def speak_result(self, text, voice_name="Dalia"):
        # Intentar primero con UWP
        if self._try_uwp_voice(text, voice_name):
            print(f"Usando voz UWP de {voice_name}")
        else:
            print(f"No se pudo usar la voz UWP de {voice_name}, usando Sabina como respaldo")
            # Usar SAPI como respaldo
            speaker = win32com.client.Dispatch("SAPI.SpVoice")
            voices = speaker.GetVoices()
            
            # Usar Sabina como respaldo
            for i in range(voices.Count):
                if "Sabina" in voices.Item(i).GetDescription():
                    speaker.Voice = voices.Item(i)
                    break
            
            speaker.Speak(text)

    def list_available_voices(self):
        ps_script = '''
        try {
            # Listar voces UWP
            $synth = New-Object Windows.Media.SpeechSynthesis.SpeechSynthesizer
            Write-Host "Voces UWP encontradas:"
            $synth.AllVoices | ForEach-Object {
                Write-Host $_.DisplayName
            }
        } catch {
            Write-Host "Error al listar voces UWP: $_"
        }

        # Listar voces SAPI
        try {
            $sapi = New-Object -ComObject SAPI.SpVoice
            Write-Host "`nVoces SAPI encontradas:"
            $sapi.GetVoices() | ForEach-Object {
                Write-Host $_.GetDescription()
            }
        } catch {
            Write-Host "Error al listar voces SAPI: $_"
        }
        '''
        
        subprocess.run(
            ["powershell", "-Command", ps_script],
            capture_output=False
        )

# Prueba
if __name__ == "__main__":
    tts = TextToSpeech()
    
    print("Listando todas las voces disponibles:")
    tts.list_available_voices()
    
    test_text = "Hola. Esta es una prueba de síntesis de voz desde python, usando las voces de windows."
    
    print("\nProbando voz de Dalia:")
    tts.speak_result(test_text, "Dalia")
    time.sleep(2)
    
    print("\nProbando voz de Jorge:")
    tts.speak_result(test_text, "Jorge")

In [ ]:
import subprocess
import win32com.client
import time

class TextToSpeech:
    def __init__(self):
        # Obtener las rutas de instalación de las voces
        self.voice_paths = self._get_voice_paths()
        
    def _get_voice_paths(self):
        ps_script = '''
        $packages = @(
            "MicrosoftWindows.Voice.es-MX.Dalia.1",
            "MicrosoftWindows.Voice.es-MX.Jorge.1"
        )
        
        foreach ($package in $packages) {
            $app = Get-AppxPackage $package
            if ($app) {
                Write-Host "$($package):$($app.InstallLocation)"
            }
        }
        '''
        
        result = subprocess.run(
            ["powershell", "-Command", ps_script],
            capture_output=True,
            text=True,
            encoding='cp1252'
        )
        
        paths = {}
        for line in result.stdout.splitlines():
            if ':' in line:
                name, path = line.split(':', 1)
                paths[name] = path.strip()
        return paths
        
    def _initialize_voice(self, voice_name):
        ps_script = f'''
        $voiceTokens = "HKCU:\\SOFTWARE\\Microsoft\\Speech_OneCore\\Voices\\Tokens"
        
        # Configurar valores según la voz
        if ("{voice_name}" -eq "Dalia") {{
            $key = "MSTTS_V110_esMX_DaliaM"
            $displayName = "Microsoft Dalia"
            $gender = "Female"
            $package = "MicrosoftWindows.Voice.es-MX.Dalia.1"
        }} else {{
            $key = "MSTTS_V110_esMX_JorgeM"
            $displayName = "Microsoft Jorge"
            $gender = "Male"
            $package = "MicrosoftWindows.Voice.es-MX.Jorge.1"
        }}
        
        $voicePath = (Get-AppxPackage $package).InstallLocation
        if ($voicePath) {{
            try {{
                # Crear o actualizar clave
                if (!(Test-Path "$voiceTokens\\$key")) {{
                    New-Item -Path "$voiceTokens\\$key" -Force | Out-Null
                }}
                
                # Configurar valores
                Set-ItemProperty -Path "$voiceTokens\\$key" -Name "Path" -Value $voicePath -Type String -Force
                Set-ItemProperty -Path "$voiceTokens\\$key" -Name "VoicePath" -Value $voicePath -Type String -Force
                Set-ItemProperty -Path "$voiceTokens\\$key" -Name "LangDataPath" -Value "$voicePath\\MSTTSLocEsMX.dat" -Type String -Force
                Set-ItemProperty -Path "$voiceTokens\\$key" -Name "Name" -Value $displayName -Type String -Force
                Set-ItemProperty -Path "$voiceTokens\\$key" -Name "Language" -Value "es-MX" -Type String -Force
                Set-ItemProperty -Path "$voiceTokens\\$key" -Name "Gender" -Value $gender -Type String -Force
                Write-Host "Voice_Init_Success"
            }} catch {{
                Write-Host "Voice_Init_Error: $_"
            }}
        }}
        '''
        
        result = subprocess.run(
            ["powershell", "-Command", ps_script],
            capture_output=True,
            text=True,
            encoding='cp1252'
        )
        return "Voice_Init_Success" in result.stdout
    
    def speak_result(self, text, voice_name="Dalia", pitch=0, rate=1, volume=100):
        # Intentar inicializar/actualizar la voz
        if self._initialize_voice(voice_name):
            try:
                speaker = win32com.client.Dispatch("SAPI.SpVoice")
                key_name = f"MSTTS_V110_esMX_{voice_name}M"
                token_path = f"HKEY_CURRENT_USER\\SOFTWARE\\Microsoft\\Speech_OneCore\\Voices\\Tokens\\{key_name}"
                
                # Intentar seleccionar la voz específica
                target_voice = None
                voices = speaker.GetVoices()
                for i in range(voices.Count):
                    if voice_name in voices.Item(i).GetDescription():
                        target_voice = voices.Item(i)
                        break
                
                if target_voice:
                    speaker.Voice = target_voice
                    print(f"Usando voz de {voice_name}")
                else:
                    print(f"No se encontró la voz de {voice_name}, usando Sabina")
                    # Usar Sabina como respaldo
                    for i in range(voices.Count):
                        if "Sabina" in voices.Item(i).GetDescription():
                            speaker.Voice = voices.Item(i)
                            break
                
                xml_text = f"""
                <pitch middle="{pitch}">
                    <rate speed="{rate}">
                        <volume level="{volume}">
                            {text}
                        </volume>
                    </rate>
                </pitch>
                """
                speaker.Speak(xml_text)
                
            except Exception as e:
                print(f"Error al usar la voz: {str(e)}")
                print("Usando Sabina como respaldo")
                self._speak_with_sabina(text, pitch, rate, volume)
        else:
            print("No se pudo inicializar la voz, usando Sabina")
            self._speak_with_sabina(text, pitch, rate, volume)
    
    def _speak_with_sabina(self, text, pitch=0, rate=1, volume=100):
        try:
            speaker = win32com.client.Dispatch("SAPI.SpVoice")
            voices = speaker.GetVoices()
            for i in range(voices.Count):
                if "Sabina" in voices.Item(i).GetDescription():
                    speaker.Voice = voices.Item(i)
                    break
                    
            xml_text = f"""
            <pitch middle="{pitch}">
                <rate speed="{rate}">
                    <volume level="{volume}">
                        {text}
                    </volume>
                </rate>
            </pitch>
            """
            speaker.Speak(xml_text)
        except Exception as e:
            print(f"Error al usar Sabina: {str(e)}")

# Prueba
if __name__ == "__main__":
    tts = TextToSpeech()
    
    test_text = "Hola. Esta es una prueba de síntesis de voz desde python, usando las voces de windows."
    
    print("\nProbando voz de Dalia:")
    tts.speak_result(test_text, "Dalia")
    time.sleep(2)
    
    print("\nProbando voz de Jorge:")
    tts.speak_result(test_text, "Jorge")

In [ ]:
import win32com.client

class TextToSpeech:
    def __init__(self, default_pitch=0, default_rate=3, default_volume=100):
        self.default_pitch = default_pitch
        self.default_rate = default_rate
        self.default_volume = default_volume
        
    def list_available_voices(self):
        try:
            speaker = win32com.client.Dispatch("SAPI.SpVoice")
            voices = speaker.GetVoices()
            print("\nVoces disponibles:")
            for i in range(voices.Count):
                print(f"{i}: {voices.Item(i).GetDescription()}")
        except Exception as e:
            print(f"Error al listar voces: {str(e)}")
    
    def speak_result(self, text, voice_name="Dalia", pitch=None, rate=None, volume=None):
        pitch = pitch if pitch is not None else self.default_pitch
        rate = rate if rate is not None else self.default_rate
        volume = volume if volume is not None else self.default_volume
        
        speaker = win32com.client.Dispatch("SAPI.SpVoice")
        voices = speaker.GetVoices()
        
        # Buscar la voz solicitada
        voice_found = False
        for i in range(voices.Count):
            if voice_name.lower() in voices.Item(i).GetDescription().lower():
                speaker.Voice = voices.Item(i)
                voice_found = True
                print(f"Usando voz de {voice_name}")
                break
        
        if not voice_found:
            print(f"No se encontró la voz de {voice_name}, usando Sabina como respaldo")
            # Usar Sabina como respaldo
            for i in range(voices.Count):
                if "Sabina" in voices.Item(i).GetDescription():
                    speaker.Voice = voices.Item(i)
                    break
        
        xml_text = f"""
        <pitch middle="{pitch}">
            <rate speed="{rate}">
                <volume level="{volume}">
                    {text}
                </volume>
            </rate>
        </pitch>
        """
        
        speaker.Speak(xml_text)

# Prueba
if __name__ == "__main__":
    tts = TextToSpeech()
    
    # Listar voces disponibles
    tts.list_available_voices()
    
    test_text = "Hola. Esta es una prueba de síntesis de voz desde python."
    
    # Probar con Dalia
    print("\nProbando voz de Dalia:")
    tts.speak_result(test_text, voice_name="Dalia")
    
    # Probar con Jorge
    print("\nProbando voz de Jorge:")
    tts.speak_result(test_text, voice_name="Jorge")

# otras pruebas mientras se reactiva claude 3.5 sonnet

In [ ]:
import pyttsx3

# Inicializar pyttsx3
engine = pyttsx3.init()

# Obtener las voces instaladas
voices = engine.getProperty('voices')

# Seleccionar la voz deseada (Dalia o Jorge)
for voice in voices:
    if "Dalia" in voice.name:  # Cambia a "Jorge" si deseas la otra voz
        engine.setProperty('voice', voice.id)

# Configurar la velocidad y el volumen
engine.setProperty('rate', 150)  # Velocidad de la voz
engine.setProperty('volume', 1.0)  # Volumen (1.0 es el máximo)

# Texto a convertir en voz
text = "Hola, esta es una prueba de la voz Dalia."

# Ejecutar el texto a voz
engine.say(text)
engine.runAndWait()


In [ ]:
import pyttsx3

engine = pyttsx3.init()
voices = engine.getProperty('voices')

for voice in voices:
    print(f"ID: {voice.id}, Name: {voice.name}")


In [ ]:
"""
Add-AppxPackage -Path "C:\Users\midna\Documents\tts\MicrosoftWindows.Voice.es-MX.Dalia.1_1.0.3.0_x64__cw5n1h2txyewy.Msix"
Add-AppxPackage -Path "C:\Users\midna\Documents\tts\MicrosoftWindows.Voice.es-MX.Jorge.1_1.0.2.0_x64__cw5n1h2txyewy.Msix"
"""

In [15]:
import os

def speak_with_narrator(text):
    command = f'powershell -Command "Add-Type –AssemblyName System.Speech; (New-Object System.Speech.Synthesis.SpeechSynthesizer).Speak(\'{text}\')"'
    os.system(command)

# Ejemplo
text = "Hola, esta es una prueba de síntesis de voz usando el narrador de Windows."
speak_with_narrator(text)


In [ ]:
import win32com.client

# Crear instancia del motor SAPI
speaker = win32com.client.Dispatch("SAPI.SpVoice")

# Obtener y listar las voces disponibles
voices = speaker.GetVoices()
print("Voces disponibles:")
for i, voice in enumerate(voices):
    print(f"{i}: {voice.GetDescription()}")


In [ ]:
import win32com.client

def speak_with_voice(text, voice_name):
    speaker = win32com.client.Dispatch("SAPI.SpVoice")
    voices = speaker.GetVoices()
    
    # Buscar la voz deseada
    for voice in voices:
        if voice_name in voice.GetDescription():
            speaker.Voice = voice
            break
    else:
        print(f"No se encontró la voz: {voice_name}")
        return

    # Configurar velocidad y volumen (opcional)
    speaker.Rate = 0  # Velocidad (0 = normal)
    speaker.Volume = 100  # Volumen (0-100)
    
    # Reproducir texto
    speaker.Speak(text)

# Probar las voces
speak_with_voice("Hola, esta es una prueba con la voz de Dalia.", "Dalia")
speak_with_voice("Hola, esta es una prueba con la voz de Jorge.", "Jorge")


In [18]:
import os

def speak_with_narrator(text):
    # Crear comando para usar el narrador
    command = f'powershell -Command "Add-Type –AssemblyName System.Speech; (New-Object System.Speech.Synthesis.SpeechSynthesizer).Speak(\'{text}\')"'
    os.system(command)

# Prueba
speak_with_narrator("Hola, esta es una prueba usando el Narrador con Dalia o Jorge.")


In [27]:
import os

def speak_with_narrator(text):
    # Escribir el texto a un archivo temporal
    with open("narrator_text.txt", "w", encoding="utf-8") as file:
        file.write(text)

    # Ejecutar el Narrador para leer el archivo
    os.system("Narrator /read narrator_text.txt")

# Prueba
speak_with_narrator("Hola, esta es una prueba con la voz de Dalia o Jorge.")


In [40]:
import os

def speak_with_narrator(text):
    # Construir el comando PowerShell para síntesis de voz
    command = f'''powershell -Command "Add-Type –AssemblyName System.Speech; ''' \
              f'''$synthesizer = New-Object System.Speech.Synthesis.SpeechSynthesizer; ''' \
              f'''$synthesizer.SelectVoice('Microsoft Dalia'); ''' \
              f'''$synthesizer.Speak('{text}');"'''

    # Ejecutar el comando
    os.system(command)

# Prueba con la voz de Dalia
speak_with_narrator("Hola. Cómo estás? Esta es una prueba con la voz de Dalia.")


In [41]:
import os

def speak_with_narrator(text):
    # Construir el comando PowerShell para síntesis de voz
    command = f'''powershell -Command "Add-Type –AssemblyName System.Speech; ''' \
              f'''$synthesizer = New-Object System.Speech.Synthesis.SpeechSynthesizer; ''' \
              f'''$synthesizer.SelectVoice('Microsoft Jorge'); ''' \
              f'''$synthesizer.Speak('{text}');"'''

    # Ejecutar el comando
    os.system(command)

# Prueba con la voz de Dalia
speak_with_narrator("Hola, esta es una prueba con la voz de Jorge.")


In [ ]:
import ctypes
from ctypes import wintypes

# Configurar las estructuras y constantes necesarias para la API de Windows
class SpeechSynthesizer(ctypes.WinDLL):
    def __init__(self):
        super().__init__('Windows.Media.SpeechSynthesis.dll')

# Inicializar el motor de síntesis
def speak_with_dalia_or_jorge(text, voice_name="Microsoft Dalia"):
    # Crear el objeto SpeechSynthesizer
    synthesizer = SpeechSynthesizer()

    # Configurar la voz
    # Aquí se asignará el nombre "Microsoft Dalia" o "Microsoft Jorge"
    synthesizer.SelectVoice(voice_name)

    # Hablar el texto
    synthesizer.Speak(text)

# Prueba
try:
    speak_with_dalia_or_jorge("Hola, esta es una prueba con la voz de Dalia.", "Microsoft Dalia")
except Exception as e:
    print(f"Error: {e}")


# Después de instalar el adapter
### NaturalVoiceSAPIAdapter
### https://github.com/gexgd0419/NaturalVoiceSAPIAdapter/releases

In [1]:
from motor_text_to_speech import speak_result

# Texto a sintetizar
result = "Hola. Probando las voces artificiales naturales, si entiendes lo que digo. Mi plaaaata hijuepuuuuuta."

# Imprimir el texto
print(result)

# Llamar a la función para sintetizar el texto
speak_result(result, pitch=0, rate=1, volume=100, voice="Microsoft Dalia")


Hola. Probando las voces artificiales naturales, si entiendes lo que digo. Mi plaaaata hijuepuuuuuta.


In [1]:
from motor_text_to_speech import speak_result

# Texto a sintetizar
result = "Hola. Probando las voces artificiales naturales, si entiendes lo que digo. Mi plaaaata hijuepuuuuuta."

# Imprimir el texto
print(result)

# Llamar a la función para sintetizar el texto
speak_result(result, pitch=0, rate=1, volume=100, voice="Microsoft Salome Online")


Hola. Probando las voces artificiales naturales, si entiendes lo que digo. Mi plaaaata hijuepuuuuuta.


In [3]:
from motor_text_to_speech import speak_result

# Texto a sintetizar
result = "Hola. Probando las voces artificiales naturales, si entiendes lo que digo. Mi plaaaata hijuepuuuuuta."

# Imprimir el texto
print(result)

# Llamar a la función para sintetizar el texto
speak_result(result, pitch=0, rate=1, volume=100, voice="Microsoft Jorge")


Hola. Probando las voces artificiales naturales, si entiendes lo que digo. Mi plaaaata hijuepuuuuuta.


In [4]:
from motor_text_to_speech import speak_result

# Texto a sintetizar
result = "Hola. Probando las voces artificiales naturales, si entiendes lo que digo. Mi plaaaata hijuepuuuuuta."

# Imprimir el texto
print(result)

# Llamar a la función para sintetizar el texto
speak_result(result, pitch=0, rate=1, volume=100, voice="Microsoft Gonzalo Online")


Hola. Probando las voces artificiales naturales, si entiendes lo que digo. Mi plaaaata hijuepuuuuuta.
